In [133]:
import os
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [307]:
compare_folder = "/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Compare/2020-11-10_COVID"
pairwise_filename = [x for x in os.listdir(compare_folder) if 'pairwise' in x][0]
pairwise_file = os.path.join(compare_folder, pairwise_filename)
pwdf = pd.read_csv(pairwise_file, sep="\t" , names=['sample_1', 'sample_2', 'dist'])

In [308]:
pwdf.head()

,sample_1,sample_2,dist
0,20069944,20069944,0
1,20069944,20069551,16
2,20069944,20091271,2
3,20069944,20081614,12
4,20069944,20267853,9


In [309]:
pwdf['sample_1'] = pwdf['sample_1'].astype(str)

In [310]:
pwdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 687241 entries, 0 to 687240
Data columns (total 3 columns):
sample_1    687241 non-null object
sample_2    687241 non-null object
dist        687241 non-null int64
dtypes: int64(1), object(2)
memory usage: 15.7+ MB


In [311]:
samples = list(set(pwdf.sample_1.tolist()))

In [312]:
len(samples)

829

In [313]:
samples_type = [type(x) for x in samples]
dict((x,samples_type.count(x)) for x in set(samples_type))

{str: 829}

In [314]:
wgs = pd.read_csv('/home/laura/ANALYSIS/MISC/microreact/WGS_COVID.csv')

In [315]:
wgs.head()

,id,full_date,patient__autocolor,province__autocolor,SEX__autocolor,AGE__autocolor,day,month,year
0,20060161,09/03/2020,174.0,Madrid,H,84,9,3,2020
1,20060178,09/03/2020,175.0,Madrid,H,76,9,3,2020
2,20062236,10/03/2020,176.0,Madrid,M,96,10,3,2020
3,20062563,10/03/2020,177.0,Madrid,H,78,10,3,2020
4,20063795,12/03/2020,178.0,Madrid,H,85,12,3,2020


In [316]:
wgs.shape

(1020, 9)

In [317]:
wgs['id'] = wgs['id'].astype(str)

In [318]:
samples_wgs = wgs.id.tolist()
#samples_wgs = [str(x) for x in samples_wgs]

In [319]:
WGS = set(samples_wgs)
PW = set(samples)

In [320]:
len(PW)

829

In [321]:
missing_wgs = list(PW - WGS)
len(missing_wgs)

47

In [322]:
missing_wgs_type = [type(x) for x in missing_wgs]
dict((x,missing_wgs_type.count(x)) for x in set(missing_wgs_type))

{str: 47}

In [323]:
missing_pw = list(WGS - PW)
len(missing_pw)

144

In [324]:
all_missing = missing_pw + missing_wgs
len(all_missing)

191

In [325]:
wgs.shape

(1020, 9)

In [326]:
df = wgs[~wgs.id.isin(missing_pw)]

In [327]:
df.shape

(870, 9)

In [328]:
df2 = pd.DataFrame(missing_wgs, columns=['id'])

In [329]:
df2['id'] = df2['id'].astype(str)

In [330]:
df2.head()

,id
0,20267855
1,20179107M
2,20174144
3,20104985
4,20095471-5


In [331]:
df.head()

,id,full_date,patient__autocolor,province__autocolor,SEX__autocolor,AGE__autocolor,day,month,year
0,20060161,09/03/2020,174.0,Madrid,H,84,9,3,2020
1,20060178,09/03/2020,175.0,Madrid,H,76,9,3,2020
2,20062236,10/03/2020,176.0,Madrid,M,96,10,3,2020
3,20062563,10/03/2020,177.0,Madrid,H,78,10,3,2020
4,20063795,12/03/2020,178.0,Madrid,H,85,12,3,2020


In [332]:
final_df = df.append(df2).reset_index(drop=True)
final_df = final_df[ ['id', 'full_date', 'patient__autocolor', 'province__autocolor', 'SEX__autocolor',
       'AGE__autocolor', 'day', 'month', 'year'] ]

In [333]:
metadata = pd.read_csv('/home/laura/ANALYSIS/MISC/microreact/wgs_covid_metadata.csv')
metadata['id'] = metadata['id'].astype(str)

In [334]:
metadata.tail()

,id,DEPARTMENT__autocolor
174,20261654,TRABAJADORES_GAP_MARANON_1
175,20298004,MANTENIMIENTO
176,20298003,TRABAJADORES_GAP_MARANON_1
177,20421509,UROLOGIA
178,20431028,REINFECCION_RUN10


In [335]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 2 columns):
id                       179 non-null object
DEPARTMENT__autocolor    179 non-null object
dtypes: object(2)
memory usage: 2.9+ KB


In [336]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 917 entries, 0 to 916
Data columns (total 9 columns):
id                     917 non-null object
full_date              870 non-null object
patient__autocolor     788 non-null float64
province__autocolor    870 non-null object
SEX__autocolor         870 non-null object
AGE__autocolor         869 non-null object
day                    870 non-null float64
month                  870 non-null float64
year                   870 non-null float64
dtypes: float64(4), object(5)
memory usage: 64.6+ KB


In [337]:
final_df = final_df.fillna(0)
final_df.day = final_df.day.astype('Int64')
final_df.month = final_df.month.astype('Int64')
final_df.year = final_df.year.astype('Int64')
final_df.replace(0, np.nan, inplace=True)

In [338]:
final_df = final_df.merge(metadata, how='outer')

In [339]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 934 entries, 0 to 933
Data columns (total 10 columns):
id                       934 non-null object
full_date                870 non-null object
patient__autocolor       788 non-null float64
province__autocolor      870 non-null object
SEX__autocolor           870 non-null object
AGE__autocolor           869 non-null object
day                      870 non-null Int64
month                    870 non-null Int64
year                     870 non-null Int64
DEPARTMENT__autocolor    212 non-null object
dtypes: Int64(3), float64(1), object(6)
memory usage: 83.0+ KB


In [340]:
final_df.tail()

,id,full_date,patient__autocolor,province__autocolor,SEX__autocolor,AGE__autocolor,day,month,year,DEPARTMENT__autocolor
929,20415500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IPR
930,20414667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IPR
931,20414667bis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IPR
932,20282399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TRABAJADORES_GAP_MARANON_1
933,20348931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TRAUMA


In [341]:
def extract_lineage(sample):
    pango_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/pangolin', str(sample) + ".lineage.csv")
    if os.path.exists(pango_file):
        df = pd.read_csv(pango_file)
        return df.lineage[0]
    else:
        return None

In [342]:
final_df['Lineage__autocolor'] = final_df.apply(lambda x: extract_lineage(x['id']), axis=1)

In [461]:
def extract_EU(sample):
    eu_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/user', str(sample) + ".tsv")
    if os.path.exists(eu_file):
        df = pd.read_csv(eu_file, sep="\t")
        if 'EU1EU2' in df.columns:
            eulist = df.EU1EU2.dropna().tolist()
        return (',').join([(x + "-" + str(eulist.count(x))) for x in set(eulist)])
    else:
        return None

In [462]:
final_df['EU1EU2__autocolor'] = final_df.apply(lambda x: extract_EU(x['id']), axis=1)

In [465]:
def extract_faulty(sample):
    sample_report = os.path.join(compare_folder, "report_samples.tsv")
    df = pd.read_csv(sample_report, sep="\t")
    df['sample'] = df['sample'].astype(str)
    if str(sample) in df['sample'].values:
        return round(df[df['sample'] == str(sample)]['faulty_frac'].values[0], 3)
    else:
        return None

In [466]:
final_df['FaultyFraction__autocolor'] = final_df.apply(lambda x: extract_faulty(x['id']), axis=1)

In [467]:
#query_dario = [20242010,20242093,20242702,20247604,20247878,20250481,20250983,20253374,20253475,20253522,20253595,20255027,20255086,20255313,20255524,20255685,20255915,20255928,20258070,20258214,20258238,20258322,20258440,20261662,20263252,20265521,20267856,20269026,20269341,20269791,20272101,20272242,20273792,20274440,20277245,20277326,20279174,20279180,20279189,20279243,20279283,20286667,20286864,20286872,20288207,20289563,20303453]
#query_dario = [str(x) for x in query_dario]


In [468]:
#final_df['Dario__autocolor'] = final_df.apply(lambda x: x['id'] in query_dario, axis=1)


In [345]:
before_june = final_df[final_df.month < 6]['id'].tolist()
before_june = [str(x) for x in before_june]
final_df['before_june_2020__autocolor'] = final_df.apply(lambda x: x['id'] in before_june, axis=1)

In [469]:
final_df = final_df.drop_duplicates(subset=['id'], keep="first")
out_file = os.path.join(compare_folder, 'wgs_microreact.csv')
final_df.to_csv(out_file, index=False)

In [470]:
final_df.head()

,id,full_date,patient__autocolor,province__autocolor,SEX__autocolor,AGE__autocolor,day,month,year,DEPARTMENT__autocolor,Lineage__autocolor,before_june_2020__autocolor,EU1EU2__autocolor,FaultyFraction__autocolor
0,20060161,09/03/2020,174.0,Madrid,H,84,9,3,2020,NaN,A,True,,0.005
1,20060178,09/03/2020,175.0,Madrid,H,76,9,3,2020,NaN,A,True,,0.004
2,20062236,10/03/2020,176.0,Madrid,M,96,10,3,2020,NaN,B.2,True,,0.001
3,20062563,10/03/2020,177.0,Madrid,H,78,10,3,2020,NaN,B.1.5,True,,0.002
4,20063795,12/03/2020,178.0,Madrid,H,85,12,3,2020,NaN,A.5,True,,0.001
